In [2]:
import networkx as nx

from OperatorBenchmark import OperatorBenchmark

from typing import List, Dict

from scripts import load_profiling_data,create_profiling_data_ER,separate_keys_and_values,git_branch_commit_push,create_profiling_data_ER_HPC

from scripts_old import load_runMultipleSimpleQWAK, load_runMultipleSimpleQWAK_legacy
from datetime import datetime

import os
import re

from utils.plotTools import plot_qwak

import numpy as np
import cupy as cp
from scipy.linalg import inv, expm
import networkx as nx
import time
import cupyx.scipy.linalg as cpx_scipy
from cupyx.profiler import benchmark
from matplotlib import pyplot as plt
import os
import json
import pickle
from tqdm import tqdm
import subprocess
import random
from qwak_cupy.qwak import QWAK as CQWAK
from qwak.qwak import QWAK as QWAK
from datetime import datetime

Profiling applied to init_operator, config: {'output_path': 'operator_results', 'output_file': None, 'tracked_attributes': ['n', 'sample', 'pVal', 'seed']}
Profiling applied to build_operator, config: {'output_path': 'operator_results', 'output_file': None, 'tracked_attributes': ['n', 'sample', 'pVal', 'seed']}
Profiling applied to build_expm_operator, config: {'output_path': 'operator_results', 'output_file': None, 'tracked_attributes': ['n', 'sample', 'pVal', 'seed']}
Profiling applied to build_multiple_operators, config: {'output_path': 'operator_anim_results', 'output_file': None, 'tracked_attributes': ['timeRange', 'sample', 'pVal', 'seed']}
Profiling applied to build_multiple_expm_operator, config: {'output_path': 'operator_anim_results', 'output_file': None, 'tracked_attributes': ['timeRange', 'sample', 'pVal', 'seed']}
Profiling applied to init_operator, config: {'output_path': 'operator_results_hpc', 'output_file': None, 'tracked_attributes': ['n', 'sample', 'pVal', 'seed']}
P

In [ ]:

nMin = 3
nMax = 1000 
n_values = list(range(nMin, nMax, 1))
pVal = 0.8
sample_range = range(0, 100, 1)
seed = 100


SCRIPT_DIR = os.getcwd()
path = os.path.normpath(os.path.join(
    SCRIPT_DIR,
    "Profiling/operator_results"
))
# Verify path exists
if not os.path.exists(path):
    print(f"ERROR: Path not found - {path}")
    print("Current directory contents:", os.listdir(SCRIPT_DIR))
    exit()
# Now load data
result = load_profiling_data(
    path=path,
    method_name="init_operator",
    nrange=n_values,
    sample_range=sample_range,  # Assuming samples 0-9
    pVal=pVal,
    seed=seed
)
result2 = load_profiling_data(
    path=path,
    method_name="build_operator",
    nrange=n_values,
    sample_range=sample_range,  # Assuming samples 0-9
    pVal=pVal,
    seed=seed
)

result3 = load_profiling_data(
    path=path,
    method_name="build_expm_operator",
    nrange=n_values,
    sample_range=sample_range,  # Assuming samples 0-9
    pVal=pVal,
    seed=seed
)

Processing n-values: 100%|██████████| 997/997 [00:49<00:00, 20.25it/s]


In [ ]:
def merge_by_sum(dict_a, dict_b):
    """
    Merge two dictionaries by summing the values of matching keys.
    Assumes both dictionaries have the same keys.
    """
    merged = {}
    for key in dict_a:
        merged[key] = dict_a[key] + dict_b[key]
    return merged


params = {
    'figsize': (12, 8),
    'plot_title' : f'Spectral Decomposition vs Expm',
    'x_label' : 'Graph Size N',
    'y_label' : "Time",
    'legend_labels' : ['spec decomp','expm'],
    'legend_loc': "best",
    'legend_title' : 'Solutions',
    'legend_ncol' : 1,
    # 'color_list' : ['#0000FF', '#008000', '#525252'],
    'color_list' : ['b','g','r'],
    'line_style_list' : ['--', '-','-.' ],
    'save_path' : f'ImgOutput/benchmark-OPERATOR.png',
    'use_loglog': False,
    'use_cbar' : False,
    'cbar_label' : None, 
    'cbar_ticks' : None,
    'cbar_tick_labels' : None,
    'x_lim' : None,
    # 'x_num_ticks' : 10,
    'y_num_ticks' : 5,
    'x_round_val' : 2,
    'y_round_val' : 3,
    # 'v_line_values' : v_line_values,
    # # 'v_line_style': '--',
    'title_font_size': 20,
    'xlabel_font_size': 22,
    'ylabel_font_size': 22,
    'legend_font_size': 14,
    'legend_title_font_size': 14,
    'tick_font_size': 18,}

merged_result = merge_by_sum(result, result2)
result_keys, result_values = separate_keys_and_values(merged_result)
# result_keys, result_values = separate_keys_and_values(result)
# result2_keys, result2_values = separate_keys_and_values(result2)
result3_keys, result3_values = separate_keys_and_values(result3)



x_value_matrix = [result_keys,result3_keys]
y_value_matrix = [result_values,result3_values]

plot_qwak(x_value_matrix = x_value_matrix, y_value_matrix = y_value_matrix,**params)

In [ ]:
nMin = 3
nMax = 1000 
n_values = list(range(nMin, nMax, 1))
pVal = 0.8
sample_range = range(0, 100, 1)
seed = 10


SCRIPT_DIR = os.getcwd()
path = os.path.normpath(os.path.join(
    SCRIPT_DIR,
    "Profiling/hiperwalk_results"
))
# Verify path exists
if not os.path.exists(path):
    print(f"ERROR: Path not found - {path}")
    print("Current directory contents:", os.listdir(SCRIPT_DIR))
    exit()
# Now load data
result_hiperwalk_init = load_profiling_data(
    path=path,
    method_name="init_hiperwalk",
    nrange=n_values,
    sample_range=sample_range,  # Assuming samples 0-9
    pVal=pVal,
    seed=seed
)
result_hiperwalk_simulate = load_profiling_data(
    path=path,
    method_name="simulate",
    nrange=n_values,
    sample_range=sample_range,  # Assuming samples 0-9
    pVal=pVal,
    seed=seed
)

Processing n-values: 100%|██████████| 997/997 [00:45<00:00, 21.96it/s]


In [ ]:
def merge_by_sum(dict_a, dict_b):
    """
    Merge two dictionaries by summing the values of matching keys.
    Assumes both dictionaries have the same keys.
    """
    merged = {}
    for key in dict_a:
        merged[key] = dict_a[key] + dict_b[key]
    return merged

params = {
    'figsize': (12, 8),
    'plot_title' : f'Spectral Decomposition vs Expm',
    'x_label' : 'Graph Size N',
    'y_label' : "Time",
    'legend_labels' : ['spec decomp','expm'],
    'legend_loc': "best",
    'legend_title' : 'Solutions',
    'legend_ncol' : 1,
    # 'color_list' : ['#0000FF', '#008000', '#525252'],
    'color_list' : ['b','g','r'],
    'line_style_list' : ['--', '-','-.' ],
    'save_path' : f'ImgOutput/benchmark-hiperwalk.png',
    'use_loglog': False,
    'use_cbar' : False,
    'cbar_label' : None, 
    'cbar_ticks' : None,
    'cbar_tick_labels' : None,
    'x_lim' : None,
    # 'x_num_ticks' : 10,
    'y_num_ticks' : 5,
    'x_round_val' : 2,
    'y_round_val' : 3,
    # 'v_line_values' : v_line_values,
    # # 'v_line_style': '--',
    'title_font_size': 20,
    'xlabel_font_size': 22,
    'ylabel_font_size': 22,
    'legend_font_size': 14,
    'legend_title_font_size': 14,
    'tick_font_size': 18,}

result_keys, result_values = separate_keys_and_values(result_hiperwalk_init)
# result_keys, result_values = separate_keys_and_values(result)
# result2_keys, result2_values = separate_keys_and_values(result2)
result3_keys, result3_values = separate_keys_and_values(result_hiperwalk_simulate)

result_hiperwalk = merge_by_sum(result_hiperwalk_init, result_hiperwalk_simulate)



x_value_matrix = [result_keys,result3_keys]
y_value_matrix = [result_values,result3_values]

plot_qwak(x_value_matrix = x_value_matrix, y_value_matrix = y_value_matrix,**params)

{3: 0.0014200000000000003, 4: 0.00127, 5: 0.0017000000000000001, 6: 0.002, 7: 0.00201, 8: 0.002, 9: 0.002, 10: 0.002, 11: 0.002, 12: 0.002, 13: 0.002, 14: 0.00202, 15: 0.0027, 16: 0.00297, 17: 0.0030299999999999997, 18: 0.003, 19: 0.003, 20: 0.003, 21: 0.003, 22: 0.00317, 23: 0.00373, 24: 0.00406, 25: 0.00404, 26: 0.00417, 27: 0.004070000000000001, 28: 0.00406, 29: 0.00451, 30: 0.005, 31: 0.00506, 32: 0.00505, 33: 0.00505, 34: 0.0058, 35: 0.006, 36: 0.00601, 37: 0.00611, 38: 0.0062, 39: 0.006960000000000001, 40: 0.007000000000000001, 41: 0.0070999999999999995, 42: 0.007000000000000001, 43: 0.007039999999999999, 44: 0.00705, 45: 0.00767, 46: 0.00801, 47: 0.00797, 48: 0.00813, 49: 0.00827, 50: 0.008369999999999999, 51: 0.008539999999999999, 52: 0.00877, 53: 0.009089999999999999, 54: 0.009139999999999999, 55: 0.00979, 56: 0.01, 57: 0.01048, 58: 0.01146, 59: 0.011399999999999999, 60: 0.01193, 61: 0.01202, 62: 0.01325, 63: 0.013080000000000001, 64: 0.01931, 65: 0.019600000000000003, 66: 0.0

In [35]:
nMin = 3
nMax = 480
n_values = list(range(nMin, nMax, 1))
pVal = 0.8
sample_range = range(0,100, 1)
seed = 10


SCRIPT_DIR = os.getcwd()
path = os.path.normpath(os.path.join(
    SCRIPT_DIR,
    "Profiling/hiperwalk_results_hpc"
))
# Verify path exists
if not os.path.exists(path):
    print(f"ERROR: Path not found - {path}")
    print("Current directory contents:", os.listdir(SCRIPT_DIR))
    exit()
# Now load data
result_hiperwalk_init_hpc = load_profiling_data(
    path=path,
    method_name="init_hiperwalk",
    nrange=n_values,
    sample_range=sample_range,  # Assuming samples 0-9
    pVal=pVal,
    seed=seed
)
result_hiperwalk_simulate_hpc = load_profiling_data(
    path=path,
    method_name="simulate",
    nrange=n_values,
    sample_range=sample_range,  # Assuming samples 0-9
    pVal=pVal,
    seed=seed
)

Processing n-values: 100%|██████████| 477/477 [00:24<00:00, 19.43it/s]


In [36]:
def merge_by_sum(dict_a, dict_b):
    """
    Merge two dictionaries by summing the values of matching keys.
    Assumes both dictionaries have the same keys.
    """
    merged = {}
    for key in dict_a:
        merged[key] = dict_a[key] + dict_b[key]
    return merged




result_hiperwalk_hpc = merge_by_sum(result_hiperwalk_init_hpc, result_hiperwalk_simulate_hpc)


In [ ]:
params = {
    'figsize': (12, 8),
    'plot_title' : f'Spectral Decomposition vs Expm',
    'x_label' : 'Graph Size N',
    'y_label' : "Time",
    'legend_labels' : ['Hiperwalk','Hiperwalk HPC', 'QWAKNP','QWAKCP'],
    'legend_loc': "best",
    'legend_title' : 'Solutions',
    'legend_ncol' : 1,
    # 'color_list' : ['#0000FF', '#008000', '#525252'],
    'color_list' : ['b','g','r'],
    'line_style_list' : ['--', '-','-.' ],
    'save_path' : f'ImgOutput/benchmark-hiperwalk.png',
    'use_loglog': False,
    'use_cbar' : False,
    'cbar_label' : None, 
    'cbar_ticks' : None,
    'cbar_tick_labels' : None,
    'x_lim' : None,
    # 'x_num_ticks' : 10,
    'y_num_ticks' : 5,
    'x_round_val' : 2,
    'y_round_val' : 3,
    # 'v_line_values' : v_line_values,
    # # 'v_line_style': '--',
    'title_font_size': 20,
    'xlabel_font_size': 22,
    'ylabel_font_size': 22,
    'legend_font_size': 14,
    'legend_title_font_size': 14,
    'tick_font_size': 18,}

# QWAK Results - Old Method

In [15]:


# Parameters
nMin = 3
nMax = 1000
nList = list(range(nMin, nMax, 1))
pVal = 0.8
samples = 100
t = 100

# base_dir = 'Datasets/Benchmark-SimpleQWAK-Test_ER2'
# base_dir = 'Datasets/Benchmark-SimpleQWAK-Test_ER2'
# base_dir_cupy = 'Datasets/Benchmark-SimpleQWAK_ER3_CuPy'

base_dir_cupy_970 = 'Profiling\Old\Benchmark-SimpleQWAK_ER-CuPy_970'
base_dir_cupy_3070 = 'Profiling\Old\Benchmark-SimpleQWAK_ER2_CuPy_3070'
base_dir_cupy_3070_2 = 'Profiling\Old\Benchmark-SimpleQWAK_ER2_CuPy_3070_2'
base_dir = 'Profiling\Old\Benchmark-SimpleQWAK_ER-NumPy'

# runMultipleSimpleQWAK(nList, pVal, t, samples, base_dir, hpc = False)
avg_list = load_runMultipleSimpleQWAK_legacy(nList, pVal,samples, t, base_dir)

avg_list_cupy_970 = load_runMultipleSimpleQWAK_legacy(nList, pVal,samples, t, base_dir_cupy_970)

avg_list_cupy_3070_2 = load_runMultipleSimpleQWAK_legacy(nList, pVal,samples, t, base_dir_cupy_3070_2)
    
# runMultipleSimpleQWAK(nList, pVal, t, samples, base_dir_cupy_3070, hpc = True)
tList_cupy_3070, qwList_cupy_3070,avg_list_cupy_3070 = load_runMultipleSimpleQWAK(nList, pVal,samples, t, base_dir_cupy_3070)

print('CuPy QWAK results computed and saved.')



Loading NPQWAK data: 100%|██████████| 997/997 [00:00<00:00, 3961.92it/s]

CuPy QWAK results computed and saved.


In [37]:

params = {
    'figsize': (12, 8),
    'plot_title' : f'QWAK NumPy vs CuPy vs Hiperwalk',
    'x_label' : 'Graph Size N',
    'y_label' : "Time",
    'legend_labels' : ['Hiperwalk','Hiperwalk HPC', 'QWAKNP','QWAKCP'],
    'legend_loc': "best",
    'legend_title' : 'Solutions',
    'legend_ncol' : 1,
    # 'color_list' : ['#0000FF', '#008000', '#525252'],
    'color_list' : ['b','g','r'],
    'line_style_list' : ['--', '-','-.' ],
    'save_path' : f'ImgOutput/benchmark-hiperwalk_hpwhpc_numpy_cupy.png',
    'use_loglog': False,
    'use_cbar' : False,
    'cbar_label' : None, 
    'cbar_ticks' : None,
    'cbar_tick_labels' : None,
    'x_lim' : None,
    # 'x_num_ticks' : 10,
    'y_num_ticks' : 5,
    'x_round_val' : 2,
    'y_round_val' : 3,
    # 'v_line_values' : v_line_values,
    # # 'v_line_style': '--',
    'title_font_size': 20,
    'xlabel_font_size': 22,
    'ylabel_font_size': 22,
    'legend_font_size': 14,
    'legend_title_font_size': 14,
    'tick_font_size': 18,}
# # Record start datetime

print(result_hiperwalk.keys())
x_value_matrix = [list(result_hiperwalk.keys()),list(result_hiperwalk_hpc.keys()),nList,nList]
y_value_matrix = [list(result_hiperwalk.values()),list(result_hiperwalk_hpc.values()),avg_list,avg_list_cupy_970]

plot_qwak(x_value_matrix = x_value_matrix, y_value_matrix = y_value_matrix,**params)

dict_keys([3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221

IndexError: list index out of range